# Data Preparation in q/kdb+

Use q to do the heavy work, handle large dataset manipulation. 

This notebook does 3 things.
1. load data
2. reshape data for visualization
3. feature engineering

# Read Data into Table

### Preview File as Text

In [1]:
fname:"/home/shiyu/Desktop/XTX_challenge/data/data-training.csv"

In [2]:
10#system "cat ",fname

"askRate0,askRate1,askRate2,askRate3,askRate4,askRate5,askRate6,askRate7,askR..
"1619.5,1620.0,1621.0,,,,,,,,,,,,,1.0,10.0,24.0,,,,,,,,,,,,,1615.0,1614.0,161..
"1619.5,1620.0,1621.0,1621.5,,,,,,,,,,,,1.0,10.0,24.0,5.0,,,,,,,,,,,,1615.0,1..
"1619.5,1620.0,1621.0,1621.5,1622.0,,,,,,,,,,,1.0,10.0,24.0,5.0,2.0,,,,,,,,,,..
"1619.5,1620.0,1621.0,1621.5,1622.0,,,,,,,,,,,1.0,10.0,24.0,5.0,22.0,,,,,,,,,..
"1619.5,1620.0,1621.0,1621.5,1622.0,,,,,,,,,,,1.0,10.0,24.0,5.0,32.0,,,,,,,,,..
"1619.5,1620.0,1621.0,1621.5,1622.0,,,,,,,,,,,1.0,10.0,24.0,5.0,152.0,,,,,,,,..
"1619.5,1620.0,1621.0,1621.5,1622.0,1623.0,,,,,,,,,,1.0,10.0,24.0,5.0,152.0,1..
"1619.5,1620.0,1621.0,1621.5,1622.0,1623.0,1623.5,,,,,,,,,1.0,10.0,24.0,5.0,1..
"1619.5,1620.0,1621.0,1621.5,1622.0,1623.0,1623.5,1624.0,,,,,,,,1.0,10.0,24.0..


### Understand the Fields

In [3]:
colnames: "," vs first system "head ",fname

show "column number: ", string count colnames
show ([]colnames:colnames)

"column number: 61"
colnames   
-----------
"askRate0" 
"askRate1" 
"askRate2" 
"askRate3" 
"askRate4" 
"askRate5" 
"askRate6" 
"askRate7" 
"askRate8" 
"askRate9" 
"askRate10"
"askRate11"
"askRate12"
"askRate13"
"askRate14"
"askSize0" 
"askSize1" 
"askSize2" 
"askSize3" 
"askSize4" 
..


In [4]:
bid_rates_col:"bidRate",/:string til 14
bid_sizes_col:"bidSize",/:string til 14
ask_rates_col:"askRate",/:string til 14
ask_sizes_col:"askSize",/:string til 14

### Load Table

In [5]:
t:(61#"F";enlist ",") 0: read0 hsym `$fname

### Add Time as Key
The data doesn't provide timestamp. We only rows are sequential ticks. Simply use row number as time to keep the sequence.

In [6]:
t: update time:i from t

[Read Data into Table] ends

# Reshape Data for Visualization Purpose

Let each row represents one level at a time, i.e. keyed by [level;time]. For visualizing orders distirbution over time.

In [7]:
/unstack data of one price level [x]
f_unstack:{?[t;();0b;`time`level`askRate`askSize`bidRate`bidSize!
                     `time,x,(`$"askRate",string x),(`$"askSize",string x),
                             (`$"bidRate",string x),(`$"bidSize",string x)]}

In [8]:
t_unstack: raze f_unstack each til 14

### Check Resulted Table

In [9]:
/check the first few lines...
10#t_unstack

time level askRate askSize bidRate bidSize
------------------------------------------
0    0     1619.5  1       1615    7      
1    0     1619.5  1       1615    7      
2    0     1619.5  1       1615    7      
3    0     1619.5  1       1615    7      
4    0     1619.5  1       1615    7      
5    0     1619.5  1       1615    7      
6    0     1619.5  1       1615    7      
7    0     1619.5  1       1615    7      
8    0     1619.5  1       1615    7      
9    0     1619.5  1       1615    7      


In [10]:
/randomly pick time points, check if bidRate for 14 levels match raw data
f_check:{14=sum value(select ifmatch:bidRate from t_unstack where time=x)
            =flip ?[t;enlist (=;`time;x);0b;(`$bid_rates_col)!`$bid_rates_col]}

rdnlist:5?count t
rdnlist!f_check peach rdnlist

       | ifmatch
------ | -------
748428 | 1      
2102632| 1      
1523210| 1      
1778945| 1      
1501018| 1      


### Dump Data into Splayed Table and .csv

The splayed table is for future loading in q. .csv is for loading in python. Right, just too rich in local storage.

In [11]:
/load function from lib
\l ../data/lib.q
system "cat ../data/lib.q"

"dumptable:{[t;fp;sep]"
" (hsym `$fp)0:sep 0:t;"
" };"


In [12]:
/save as splayed table
`:/home/shiyu/Desktop/XTX_challenge/data/data_unstack_splayed/ set t
/ save as .csv
dumptable[t_unstack;"/home/shiyu/Desktop/XTX_challenge/data/data_unstack.csv";","]

`:/home/shiyu/Desktop/XTX_challenge/data/data_unstack_splayed/


[Reshape Data for Visualization Purpose] ends

# Feature Engineering

In [7]:
/fill null
t:0^t

In [8]:
/utilities
/diff function
f_diff:{[col;lag]l:count t;(lag#0f),(((lag-l)#t[col])-(l-lag)#t[col])} /return list
f_diff_t:{(enlist`$x,"_diff",string y)!enlist f_diff[`$x;y]} /return table

### Mid Price, Spread, Book Price
... and its Moving Average, Moving Volatility and Change

In [9]:
/mid price = (askRate0+bidRate0)/2
t:update midprice:(askRate0+bidRate0)%2 from t

/spread = askRate0 - bidRate0
t:![t;();0b;(enlist `spread)!enlist (-;`askRate0;`bidRate0)]

/book price = (askRate_i*bidSize_i+bidRate_i*askSize_i)/(askSize_i+bidSize_i)
f_bp:{[aR;aS;bR;bS](aR*bS+bR*aS)%(aS+bS)}
t:![t;();0b;raze{(enlist `$"bookprice",string x)!
                  enlist(f_bp;
                         `$"askRate",string x;
                         `$"askSize",string x;
                         `$"bidRate",string x;
                         `$"bidSize",string x)} peach til 3]

In [10]:
/moving average of prices/spread

/col: column name
/l: window length
f_ma_t:{[col;l](enlist `$(string col),"_ma",string l)!enlist (l msum t col)%l }
t:![t;();0b;raze raze (f_ma_t each (`midprice`spread`bookprice0)) peach (5 15 45 87 180 360)]

In [11]:
/moving volatility of prices/spread
f_std_t:{[col;l](enlist `$(string col),"_std",string l)!enlist (l msum t col)%l }
t:![t;();0b;raze raze (f_std_t each (`midprice`spread`bookprice0)) peach (5 15 45 87 180 360)]

In [12]:
/change of prices/spread
t:![t;();0b;raze raze (f_diff_t each string `midprice`spread`bookprice0) peach (5 15 45 87 180 360)]

### Volume, Level
... and its Change

In [13]:
/level change
t:![t;();0b;raze raze (f_diff_t each (ask_rates_col,bid_rates_col)) peach (5 15 45 87 180 360)]

/volume = sum of ask/bid size for level 0-14
t:![t;();0b;(enlist `askSize_ttl)!enlist sum?[t;();();(`$ask_sizes_col)!`$ask_sizes_col]]
t:![t;();0b;(enlist `bidSize_ttl)!enlist sum?[t;();();(`$bid_sizes_col)!`$bid_sizes_col]]

/volume change
t:![t;();0b;raze raze (f_diff_t each (ask_sizes_col,bid_sizes_col)) peach (5 15 45 87 180 360)]

### Volume Imbalance
... and its Change

In [14]:
/volume imbalance for level i = bidSize_i/(bidSize_i+askSize_i)
f_vol_imb:{[aS;bS]bS%(aS+bS)}
t:![t;();0b;raze{(enlist `$"vol_imb",string x)!
                  enlist (f_vol_imb;
                          `$"askSize",string x;
                          `$"bidSize",string x)} peach til 15]

In [15]:
/aggregated volume imbalance for level i-j

/tt: table
/i: start level
/j: end level
/d: direction {ask,bid}
f_vol_agg:{[tt;i;j;d]col:d,/:"Size",/:string(i-j-1)#til j+1; /columns to aggregate
            sum?[tt;();();(`$col)!`$col]} /return aggregated column
f_vol_imb_agg:{[tt;i;j]f_vol_imb[f_vol_agg[tt;i;j;"ask"];f_vol_agg[tt;i;j;"bid"]]}

/level 0-4
t:![t;();0b;(enlist `$"vol_imb_agg","0_4")!enlist f_vol_imb_agg[t;0;4]]
/level 5-14
t:![t;();0b;(enlist `$"vol_imb_agg","5_14")!enlist f_vol_imb_agg[t;5;14]]

In [16]:
/aggregated volume imbalance change
t:![t;();0b;raze raze (f_diff_t each string `vol_imb_agg0_4`vol_imb_agg5_14) peach (5 15 45 87 180 360)]

### Level Sparsity

In [17]:
/level sparsity = askRate_i - askRate_0
/d: direction {ask/bid}
/l: level 1-14
f_spar:{[d;l]col:d,"Rate";t[`$col,string l]-t[`$col,string 0]}
f_spar_t:{[d;l](enlist`$ raze d,"Rate",(string l),"_spa")!enlist f_spar[d;l]}

t:![t;();0b;raze raze (f_spar_t each ("ask";"bid")) peach (1,2,3,7,14)]

/aggregated clipped top 3 level sparsity
bid_diff: sum 0|2.5&-[t `$"bidRate",/:string til 3;t `$"bidRate",/:string -3#til 4]-0.5
ask_diff: sum 0|2.5&-[t `$"askRate",/:string -3#til 4;t `$"askRate",/:string til 3]-0.5
t:![t;();0b;(enlist `bard)!enlist (bid_diff+ask_diff)]

### Dump Table of Features into Splayed Table

In [18]:
`:/home/shiyu/Desktop/XTX_challenge/data/data_features_splayed/ set t

`:/home/shiyu/Desktop/XTX_challenge/data/data_features_splayed/


[Feature Engineering] ends